In [41]:
import os,sys
sys.path.append('../..')
sys.path.append('../../../gempy/')
import sandbox.sandbox as sb

import gempy as gp
#import gempy_sandbox a   s gpsb
import numpy as np
#from PIL import Image, ImageDraw
import pickle

### Set up the sandbox 

In [38]:
calibration_filename = "sandbox_test.dat" 

kinect = sb.Kinect(dummy=True)

dummy mode. get_frame() will return a synthetic depth frame, other functions may not work


In [39]:
projector = sb.Projector(resolution = (1024,768))
projector.work_directory ="./temp"
projector.refresh = 100

projector.input_rescale  = False  
calibration = projector.calibration
calibration.load(calibration_file=calibration_filename)

#projector.start_stream()          #Open an new Tab with sandbox output.

created new calibration: <sandbox.sandbox.Calibration object at 0x1c40ab01d0>


### Setup the model

In [40]:

geo_model = gp.load_model('./../../../gempy/notebooks/tutorials/saves/ch1-2.pickle')
scale=sb.Scale(extent=[0,2000,0,1500,0,2000] , calibration=calibration) #initialize scale object to handle the scaling of the model in the andbox

scale.calculate_scales()

grid = sb.Grid(calibration=calibration,scale=scale)

Aspect ratio of the model is fixed in XY
Model size is limited by X dimension
scale in Model units/ mm (X,Y,Z): [2.0, 2.5, 2.092050209205021]
the shown extent is [0.0, 1996.875, 0.0, 1496.875] 


In [5]:
!ls ./../../../

2016-ml-contest broken_gempy    gempy           libfreenect     open_AR_Sandbox
AR-Sandbox      devito          gempy-server    old_gempy       sandbox


### set up the visualization

In [6]:
x = range(scale.output_res[0])
y = range(scale.output_res[1])

geol_map = sb.Plot(
    calibration=calibration,
    cmap=gp.plot.cmap,
    norm=gp.plot.norm
    )


fault_line = sb.Contour(
    start=0.5,
    end=2.5,
    step=1.0,
    linewidth=3.0,
    colors=[(1.0, 1.0, 1.0, 1.0)]
    )

main_contours = sb.Contour(
    start=scale.extent[4],
    end=scale.extent[5],
    step=100,
    show_labels=True
    )

sub_contours = sb.Contour(     

    start=scale.extent[4],
    end=scale.extent[5],
    step=25,
    linewidth=0.8,
    colors=[(0, 0, 0, 0.7)],
   )



 

NameError: name 'sol' is not defined

In [32]:
print()

<bound method Normalize.scaled of <matplotlib.colors.BoundaryNorm object at 0x18192f1eb8>>


In [22]:
#new structure
def run():
    grid.update_grid(kinect.get_filtered_frame(n_frames=3,sigma_gauss=4))
    sol = gp.compute_model_at(grid.depth_grid, geo_model)

    lith_block = sol.lith_block.reshape((scale.output_res[1], scale.output_res[0]))
    scalar_block = sol.scalar_field_lith.reshape((scale.output_res[1], scale.output_res[0]))
    geol_map.create_empty_frame()
    #geol_map.render_frame(lith_block)
    lith_levels = sol.scalar_field_at_interfaces[-1].sort()
    geol_map.add_lith_contours(scalar_block, levels=lith_levels)
    
    elevation = grid.depth_grid.reshape((scale.output_res[1], scale.output_res[0], 3))[:, :, 2]
    #fault_data = sol.fault_blocks.reshape((scalgeol_map.outfilee.output_res[1], scale.output_res[0])) 
    for fault in sol.fault_blocks:
        fault=fault.reshape((scale.output_res[1],scale.output_res[0]))
        geol_map.add_contours(fault_line, [x,y,fault])


    geol_map.add_contours(main_contours,[x,y,elevation])
    geol_map.add_contours(sub_contours,[x,y,elevation])
    geol_map.save(outfile="./temp/current_frame.png")
    geol_map.save(outfile="./temp/current_frame.pdf")   
    projector.show(rescale=False)

In [23]:
while True:
run()


[[30.73383904 30.73184204 30.72902298 ... 30.80283928 30.7990818
  30.79532433]
 [30.73384285 30.73184586 30.7290287  ... 30.80280304 30.79904556
  30.79528427]
 [30.73384476 30.73184967 30.72903252 ... 30.8027668  30.79900742
  30.79524612]
 ...
 [30.33140755 30.72424698 30.72144127 ... 30.8018837  30.79789543
  30.79390335]
 [30.33158112 30.7245121  30.72171021 ... 30.80216408 30.79817581
  30.79418564]
 [30.33175278 30.72477913 30.72197914 ... 30.31230736 30.31201935
  30.31172943]]
[[3. 3. 3. ... 2. 2. 2.]
 [3. 3. 3. ... 2. 2. 2.]
 [3. 3. 3. ... 2. 2. 2.]
 ...
 [6. 3. 3. ... 2. 2. 2.]
 [6. 3. 3. ... 2. 2. 2.]
 [6. 3. 3. ... 6. 6. 6.]]


In [33]:
def export_topographic_map(output="./temp/topographic_map.pdf"):
    grid.update_grid(kinect.get_filtered_frame(n_frames=3,sigma_gauss=4))
    geol_map.create_empty_frame()
    elevation = grid.depth_grid.reshape((scale.output_res[1], scale.output_res[0], 3))[:, :, 2]
    geol_map.add_contours(main_contours,[x,y,elevation])
    geol_map.add_contours(sub_contours,[x,y,elevation])
    geol_map.save(outfile=output)  
    
def export_geological_map(output="./temp/geological_map.pdf"): 
    print("there is still a bug in the map that causes the uppermost lithology to be displayed in the basement color. Unfortunately we do not have a quick fix for this currently... Sorry! Please fix the map yourself, for example using illustrator")
    grid.update_grid(kinect.get_filtered_frame(n_frames=3,sigma_gauss=4))
    sol = gp.compute_model_at(grid.depth_grid, geo_model)
    
    lith_block = sol.lith_block.reshape((scale.output_res[1], scale.output_res[0]))
    geol_map.create_empty_frame()
    lith_levels = sol.scalar_field_at_interfaces[-1].sort()
    geol_map.add_lith_contours(lith_block, levels=lith_levels)
    
    elevation = grid.depth_grid.reshape((scale.output_res[1], scale.output_res[0], 3))[:, :, 2]
    #fault_data = sol.fault_blocks.reshape((scalgeol_map.outfilee.output_res[1], scale.output_res[0])) 
    for fault in sol.fault_blocks:
        fault=fault.reshape((scale.output_res[1],scale.output_res[0]))
        geol_map.add_contours(fault_line, [x,y,fault])
        
    geol_map.add_contours(main_contours,[x,y,elevation])
    geol_map.add_contours(sub_contours,[x,y,elevation])
    geol_map.save(outfile=output)  



In [34]:
projector.start_stream()

In [35]:
export_geological_map()

there is still a bug in the map that causes the uppermost lithology to be displayed in the basement color. Unfortunately we do not have a quick fix for this currently... Sorry! Please fix the map yourself, for example using illustrator


In [36]:
export_topographic_map()

x

## deprecated:

In [ ]:
kinect.sigma_gauss=7

### Toggle fault visibility

In [ ]:
geology.pause()
if geology.module.show_faults is True:
    geology.module.show_faults = False
else:
    geology.module.show_faults = True
geology.resume()
    

## change contour levels 

In [ ]:
       self.fig = plt.figure(figsize=(self.w, self.h), dpi=100, frameon=False)
        self.ax = plt.Axes(self.fig, [0., 0., 1., 1.])
        self.ax.set_axis_off()
        self.fig.add_axes(self.ax)

geology.pause()
start = 0
end = 2000
main_step = 50
sub_step = 10

geology.module.main_contours = np.arange(start, end, main_step)
geology.module.sub_contours = np.arange(start, end, sub_step)
geology.resume()

In [ ]:
geology.lock.acquire()
geology.lock.release()


In [ ]:
geo_model_3d = gp.load_model('./gempy/notebooks/tutorials/saves/ch1-2.pickle')


In [ ]:
for i in range(10):
    with geology.lock:
        gp.plot.plot_section(geo_model_3d, 10)

In [ ]:
with geology.lock:
    for i in range(10):
        gp.plot.plot_section(geo_model_3d, 10)

In [ ]:
geology.lock.release()

In [ ]:
geology.lock.locked()

In [ ]:
geology.run()

In [ ]:
a = 0
for i in range(100000):
    a += i


In [ ]:
with geology.lock:
    for i in range(10):
        gp.compute_model(geo_model_3d)

In [ ]:
geo_model.interpolator.theano_graph

In [ ]:
for i in range(10):
    gp.compute_model(geo_model_3d)

In [ ]:
geo_model_3d.solutions